# ***Automatic Neuron Tracking System for Unconstrained Nematodes (ANTSUN) v2.2.1***

This pipeline takes as input `.nd2` files from the confocal scope corresponding to freely-moving and immobilized recordings for NeuroPAL, together with an `.h5` file from the NIR tracking camera. It processes the confocal data to locate and track neurons in the worm's head, and thereby generates GCaMP traces for these neurons. It also generates the NeuroPAL RGB image used for labeling, labels it with AutoCellLabeler, and maps the labels back to the freely-moving traces. It also processes the NIR data to extract behavioral variables that can be correlated with the calcium traces.

The expected runtime of this pipeline is 34 hours.


## General notes

### GPU acceleration

Many steps of `ANTSUN` are GPU-accelerated (they are labeled as such beforehand). Thus, you need to ensure that there is sufficient GPU memory on your device before running those steps. It is recommended to have at least 8GB available before running any GPU-accelerated step. You can view how much GPU memory is available by running `nvidia-smi`.

- On `flv-c1`, there is only one GPU with 8GB memory. ANTSUN will likely fail as this is not enough memory.
- On `flv-c2`, there are four GPUs, two with 16GB memory each and two with 24GB each.
- On `flv-c3`, there are four GPUs, three with 24GB memory each and one with 48GB.
- On `flv-c4`, there are four GPUs, each with 48GB.

If you're using a server with more than one GPU, you can specify which one to use (according to its index in `nvidia-smi` by setting `nvidia_smi_device` and `flv_c` appropriately). Please reserve the GPU using our [GPU reservations sheet](https://docs.google.com/spreadsheets/d/1wMQ_bGJly15_1CXspK0fWzs-bd-lF-tVMpOVZZZ6SAc/edit?gid=922051617#gid=922051617).

**Please restart the kernel after the notebook is done running.**

### Error recovery

If a step of the pipeline encounters an unexpected error, it is usually advisable to restart the kernel. **WARNING:** Restarting the kernel while save operations are in progress could corrupt the data files and possibly mean you would have to start over from the very beginning. Thus, before restarting the kernel, you should check that `JLD2` files are not actively being saved, and only restart the kernel once the pipeline is on a step other than saving them.

After you restart the kernel, rerun the "Load packages", "Data parameters" and "Initialize data dictionaries" steps of the pipeline. This should reset the pipeline into its last usable state. If the error was due to running out of memory or some other hardware issue, assuming there are computing resources available, you should then be able to run the pipeline from the step that generated the error through the end.

If the error was due to you making a mistake in the "Data parameters" section of the pipeline, you may need to rerun the "Pipeline parameters" and "Save parameter settings" sections of the notebook after running the "Initialize data dictionaries" step, but note that you should **not** rerun the "Read meta-parameters from the ND2 file" step.

In [2]:
nvidia_smi_device = 0
# flv_c = X

0

In [3]:
# Expand the path to include all Conda binary locations
conda_env = "C:\\Users\\munib\\miniconda3\\envs\\nir-pipeline"
new_paths = [
    joinpath(conda_env, "bin"),
    joinpath(conda_env, "Library", "bin"),
    joinpath(conda_env, "Scripts")
]
ENV["PATH"] = join(new_paths, ";") * ";" * ENV["PATH"]

using PyCall
# Re-verify PyCall is definitely looking at the right python
println("PyCall using: ", PyCall.python) 

torch = pyimport("torch")
using Pkg
ENV["PYTHON"] = "C:\\Users\\munib\\miniconda3\\envs\\nir-pipeline\\python.exe"
Pkg.build("PyCall")
using PyCall
PyCall.python
# make sure this is the correct path to your conda environment's python, which should have all the necessary packages installed.
torch = pyimport("torch")

println("Is CUDA available? ", torch.cuda.is_available())
println("GPU Name: ", torch.cuda.get_device_name(0))

PyCall using: C:\Users\munib\miniconda3\envs\nir-pipeline\python.exe


    Building Conda ─→ `C:\Users\munib\.julia-flv\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\8f06b0cfa4c514c7b9546756dbae91fcfbc92dc9\build.log`
    Building PyCall → `C:\Users\munib\.julia-flv\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\9816a3826b0ebf49ab4926e2b18842ad8b5c8f04\build.log`


Is CUDA available? true
GPU Name: NVIDIA GeForce RTX 5080


In [4]:
gpu_device = torch.device("cuda:$nvidia_smi_device")
print("Using GPU: ", gpu_device, torch.cuda.get_device_name(gpu_device))

Using GPU: PyObject device(type='cuda', index=0)NVIDIA GeForce RTX 5080

In [5]:
ENV["CUDA_VISIBLE_DEVICES"] = "$(gpu_device)"
ENV["CUDA_VISIBLE_DEVICES"] = "0"   # Use GPU 0; set to "" for CPU
ENV["JULIA_IO_BUFFER"] = "0"

"0"

## Load packages

In [6]:
# Flavell lab packages
using ND2Process
using GPUFilter
# using WormFeatureDetector # not installed properly
using NRRDIO
using FlavellBase
# using SegmentationTools # not installed properly
using ImageDataIO
# using RegistrationGraph # not installed properly
# using ExtractRegisteredData # not installed properly
# using CaAnalysis # not installed properly
using BehaviorDataNIR
using UNet2D
# using ImageRegistration # not installed properly

# Other packages
using ProgressMeter
using PyCall
using PyPlot
using Statistics
using StatsBase
using DelimitedFiles
using Images
using Cairo
using Distributions
using DataStructures
using HDF5
using Interact
using WebIO
# using Plots # not installed properly
# using GraphPlot # was already commented out in the original code
# using LightGraphs # was already commented out in the original code
# using SimpleWeightedGraphs # was already commented out in the original code
using Dates
using JLD2
using TotalVariation
using VideoIO
using Distributions
using MultivariateStats
using FFTW
using LinearAlgebra
using GLMNet
using InformationMeasures
using CUDA
using LsqFit
using Optim
using Rotations
using CoordinateTransformations
using ImageTransformations
using Interpolations
using H5Zblosc

using Distributed

┌ Warning: CUDA runtime library cudart64_12.dll was loaded from a system path. This may cause errors.
│ Ensure that you have not set the LD_LIBRARY_PATH environment variable, or that it does not contain paths to CUDA libraries.
└ @ CUDA C:\Users\munib\.julia-flv\packages\CUDA\rXson\src\initialization.jl:189
┌ Warning: CUDA runtime library caffe2_nvrtc.dll was loaded from a system path. This may cause errors.
│ Ensure that you have not set the LD_LIBRARY_PATH environment variable, or that it does not contain paths to CUDA libraries.
└ @ CUDA C:\Users\munib\.julia-flv\packages\CUDA\rXson\src\initialization.jl:189
┌ Warning: CUDA runtime library nvrtc64_120_0.dll was loaded from a system path. This may cause errors.
│ Ensure that you have not set the LD_LIBRARY_PATH environment variable, or that it does not contain paths to CUDA libraries.
└ @ CUDA C:\Users\munib\.julia-flv\packages\CUDA\rXson\src\initialization.jl:189
┌ Warning: CUDA runtime library cublas64_12.dll was loaded from a syst

WebIO._IJuliaInit()

# Parameter settings

Expected runtime: 1 minute

### Data parameters

This section includes dataset-specific parameters, including the filename, voxel spacing, laser intensity step-up, and channel identities. It must be run every time you initialize the pipeline. 

**Important:** If you already have dictionaries saved and are rerunning the pipeline, skip to "Initialize Data Dictionaries" after running this section, and do not rerun the "Pipeline parameters" section unless you have modified something here.

In [ ]:
data_dir = "C:\\Users\\munib\\POSTDOC\\DATA\\g5ht-free"
dataset = "date-20260113_strain-ISg5HT-ADF-TeTx_condition-starvedpatch_worm004"

# extract date from dataset name
datestr = split(dataset, "_")[1] |> x -> split(x, "-")[2]
output_dir = joinpath(data_dir, datestr, dataset)
path_h5 = output_dir * ".h5"
path_nd2 =  output_dir * ".nd2"

println("HDF5 file path: ", path_h5)
println("ND2 file path: ", path_nd2)

HDF5 file path: C:\Users\munib\POSTDOC\DATA\g5ht-free\20260113\date-20260113_strain-ISg5HT-ADF-TeTx_condition-starvedpatch_worm004.h5
ND2 file path: C:\Users\munib\POSTDOC\DATA\g5ht-free\20260113\date-20260113_strain-ISg5HT-ADF-TeTx_condition-starvedpatch_worm004.nd2


In [21]:
# ----------------------
# Globals
# ----------------------
ch_bluegreen = 1 # g5-HT3.0
ch_red = 2 # mCherry
h5_confocal_time_lag = 0 # if this is the second confocal dataset using the same h5 file, set equal to number of frames in 1st dataset
spacing_lat = 0.54 # Spacing of each voxel in the xy plane. If different, it will likely be necessary to modify pipeline parameters.
spacing_axi = 1.08 # Spacing of each voxel in the z dimension. If different, it will likely be necessary to modify pipeline parameters.
n_z = 41 # Number of z-slices per z-stack
z_range = 3:41 # Which z-slices to use. We exclude the first two due to piezo motion artifacts.
n_rec = 1

# ----------------------
# Params
# ----------------------
param = Dict()
param["freely_moving"] = Dict()

param["n_rec"] = n_rec


# ----------------------
# Paths
# ----------------------
param_paths = Dict()
param_paths["freely_moving"] = Dict()
param_paths["freely_moving"]["path_h5"] = path_h5
param_paths["freely_moving"]["path_nd2"] = path_nd2

# ----------------------
# Data dict
# ----------------------
data_dicts = Dict()
data_dicts["freely_moving"] = Dict()

Dict{Any, Any}()

### Sync NIR to confocal, find heat-stims

Expected runtime: 10 seconds.

This section matches timestamps between the confocal and NIR videos, so that neural activity and behavior can be matched across time.

If this section crashes, it is very likely that your dataset is unusable.

In [ ]:
get_timing_info!(data_dicts["freely_moving"], param["freely_moving"], param_paths["freely_moving"]["path_h5"], h5_confocal_time_lag);

LoadError: UndefVarError: data_dicts not defined

In [48]:
data_dict["confocal_to_nir"], data_dict["nir_to_confocal"], data_dict["timing_stack"], data_dict["timing_nir"] = sync_timing(path_h5, param["n_rec"]);

LoadError: BoundsError: attempt to access 0-element Vector{Int64} at index [0]

In [23]:
n_img_nir, daqmx_ai, daqmx_di, img_metadata = h5open(path_h5, "r") do h5f
    n_img_nir = size(h5f["img_nir"])[3]
    daqmx_ai = read(h5f, "daqmx_ai")
    daqmx_di = read(h5f, "daqmx_di")
    img_metadata = read(h5f, "img_metadata")
    n_img_nir, daqmx_ai, daqmx_di, img_metadata
end

(9432, [0.998726536568775 4.993080962732966 0.0026847333741240598; 1.0010268636374178 4.994395436834542 0.0026847333741240598; … ; 0.9993837728742496 4.992423725682724 0.0026847333741240598; 0.9993837728742496 4.9927523442077995 0.0023561150859444935], UInt32[0x00000000 0x00000002; 0x00000000 0x00000002; … ; 0x00000000 0x00000002; 0x00000000 0x00000000], Dict{String, Any}("q_recording" => Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "img_timestamp" => [3714400320873512, 3714400470878376, 3714400495878376, 3714400520879728, 3714400545880704, 3714400570880728, 3714400595881864, 3714400620882248, 3714400645882920, 3714400720885760  …  3714879785928408, 3714879810928864, 3714879835929376, 3714879860930728, 3714879885930784, 3714879910931864, 3714879935932808, 3714879960933080, 3714879985934352, 3714880010934864], "img_id" => [61447314, 61447320, 61447321, 61447322, 61447323, 61447324, 61447325, 61447326, 61447327, 61447330  …  61466492, 61466493, 61466494, 61466495,

In [31]:
function filter_ai_laser(ai_laser, di_camera, n_rec=1)
    n_ai, n_di = length(ai_laser), length(di_camera)
    n = min(n_ai, n_di)
    ai_laser_zstack_only = Float32.(ai_laser[1:n])
    ai_laser_filter_bit = zeros(Float32, n)
    trg_state = zeros(Float64, n)

    n_y = n
    n_kernel = 100
    @simd for i = 1:n_y
        start = max(1, i - n_kernel)
        stop = min(n_y, i + n_kernel)

        trg_state[i] = maximum(di_camera[start:stop])
    end

    Δtrg_state = diff(trg_state)    
    list_idx_start = findall(Δtrg_state .== 1)
    list_idx_end = findall(Δtrg_state .== -1)

    if n_rec > length(list_idx_start)
        error("filter_ai_laser: not enough recording detected. check n_rec")
    end
    
    list_idx_rec = sortperm(list_idx_end .- list_idx_start, rev=true)[1:n_rec]
    for i = list_idx_rec
        ai_laser_filter_bit[list_idx_start[i]+1:list_idx_end[i]-1] .= 1
    end
    
    ai_laser_filter_bit .* ai_laser_zstack_only
end

filter_ai_laser (generic function with 2 methods)

In [ ]:
daqmx_ai

2400520×3 Matrix{Float64}:
 0.998727  4.99308  0.00268473
 1.00103   4.9944   0.00268473
 0.999712  4.99505  0.00235612
 1.00103   4.99538  0.00268473
 0.999712  4.99735  0.00268473
 0.999055  4.99472  0.00268473
 1.00004   4.98717  0.00235612
 1.0007    4.99702  0.00235612
 0.999384  4.99144  0.00235612
 0.999384  4.99242  0.00235612
 1.00004   4.99242  0.00268473
 0.999384  4.99407  0.00268473
 1.0007    4.99177  0.00235612
 ⋮                  
 0.999712  4.99407  0.00268473
 0.999055  4.99242  0.00268473
 1.00037   4.99242  0.00235612
 1.0007    4.99407  0.00268473
 0.999384  4.99045  0.00235612
 0.998069  4.99045  0.0020275
 0.998727  4.9921   0.00268473
 0.999712  4.99472  0.00268473
 0.999712  4.98749  0.00235612
 1.00037   4.99177  0.00301335
 0.999384  4.99242  0.00268473
 0.999384  4.99275  0.00235612

In [47]:
ai_laser = filter_ai_laser(daqmx_ai[:,1], daqmx_di[:,1], n_rec)
ai_laser[10000:30000]

20001-element Vector{Float32}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 1.000041
 1.0003697
 0.9997124
 1.0003697
 1.0006982
 1.000041
 1.0020127
 0.9997124
 1.0013555
 1.0006982
 1.0020127
 0.9997124

In [46]:
# how many elements in ai laser equal 0.0 and are greather than 0
println(sum(ai_laser .== 0.0))
println(sum(ai_laser .> 0.0))

46245
2354220
